In [1]:
import pandas as pd
import plotly.express as px
import numpy as np

In [6]:

deaths = pd.read_csv('deaths-drug-overdoses.csv')
dr_all = pd.read_csv('death-rates-from-drug-use-disorders.csv')

In [7]:
dr_all.rename(columns={'Deaths - Drug use disorders - Sex: Both - Age: Age-standardized (Rate)':'deaths'}, inplace=True)
dr_all.head(5)

,Entity,Code,Year,deaths
0,Afghanistan,AFG,1990,1.17
1,Afghanistan,AFG,1991,1.18
2,Afghanistan,AFG,1992,1.21
3,Afghanistan,AFG,1993,1.25
4,Afghanistan,AFG,1994,1.30


In [8]:
dr_all['deaths'].max()

18.83

In [9]:
deaths.head()

,Entity,Code,Year,Deaths - Opioid use disorders - Sex: Both - Age: All Ages (Number),Deaths - Cocaine use disorders - Sex: Both - Age: All Ages (Number),Deaths - Other drug use disorders - Sex: Both - Age: All Ages (Number),Deaths - Amphetamine use disorders - Sex: Both - Age: All Ages (Number)
0,Afghanistan,AFG,1990,73,3,15,2
1,Afghanistan,AFG,1991,80,3,16,2
2,Afghanistan,AFG,1992,93,4,19,3
3,Afghanistan,AFG,1993,104,4,21,3
4,Afghanistan,AFG,1994,112,4,22,3


In [10]:
deaths_colnames = {
    'Deaths - Opioid use disorders - Sex: Both - Age: All Ages (Number)' : "Opioid",
    'Deaths - Cocaine use disorders - Sex: Both - Age: All Ages (Number)' : "Cocaine",
    'Deaths - Other drug use disorders - Sex: Both - Age: All Ages (Number)' : "Other",
    'Deaths - Amphetamine use disorders - Sex: Both - Age: All Ages (Number)' : "Amphetamine"}
deaths.rename(columns=deaths_colnames, inplace=True)


In [11]:
euro_countries = ['Austria','Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czechia',
                  'Denmark','Estonia','Finland','France','Germany','Greece','Hungary',
                  'Ireland','Italy','Latvia','Lithuania','Luxembourg','Malta','Netherlands',
                  'Poland','Portugal','Romania','Slovakia','Slovenia','Spain','Sweden']

In [12]:
len(euro_countries)

27

In [13]:
xdeaths = deaths.reset_index()
deaths = pd.melt(xdeaths, id_vars=['Entity', 'Code', 'Year'], value_vars=['Opioid', 'Cocaine', 'Other','Amphetamine'])

In [14]:
deaths = deaths.rename(columns={'variable': 'drug',
                                'value':'total'})
deaths['euro'] = deaths["Entity"].isin(euro_countries) * 1

In [10]:
deaths

,Entity,Code,Year,drug,total,euro
0,Afghanistan,AFG,1990,Opioid,73,0
1,Afghanistan,AFG,1991,Opioid,80,0
2,Afghanistan,AFG,1992,Opioid,93,0
3,Afghanistan,AFG,1993,Opioid,104,0
4,Afghanistan,AFG,1994,Opioid,112,0
...,...,...,...,...,...,...
27355,Zimbabwe,ZWE,2015,Amphetamine,7,0
27356,Zimbabwe,ZWE,2016,Amphetamine,7,0
27357,Zimbabwe,ZWE,2017,Amphetamine,8,0
27358,Zimbabwe,ZWE,2018,Amphetamine,8,0


In [17]:
death_sub = deaths.groupby(['drug'])['total'].sum().reset_index()
death_sub = death_sub.sort_values('total',ascending=True)

In [49]:
death_sub = deaths[deaths['drug'] == "Opioid"]
death_sub = death_sub.groupby(['Entity'])['total'].sum().reset_index()
death_sub = death_sub.sort_values('total',ascending=True).tail(8)

In [16]:
death_sub

,drug,total
1,Cocaine,1212607
0,Amphetamine,1404294
3,Other,2550514
2,Opioid,11583657


In [34]:
fig = px.bar(death_sub,
       x="total",
       y='Entity',
       orientation='h')

fig.show()

In [43]:
data_slider = []
data_each_yr = dict(type='choropleth',
                        locations = death_sub['Entity'],
                        locationmode='country names',
                        autocolorscale = False,
                        z=np.log(death_sub['total'].astype(float)),
                        zmin=0, 
                        zmax = np.log(deaths[deaths['drug']== "Opioid"]['total'].max()),
                        colorscale = ["#ffe2bd", "#006837"],   
                        marker_line_color= 'rgba(0,0,0,0)',
                        colorbar= {'title':'Tonnes (log)'},#Tonnes in logscale
                        colorbar_lenmode='fraction',
                        colorbar_len=0.8,
                        colorbar_x=1,
                        colorbar_xanchor='left',
                        colorbar_y=0.5,
                        name='')
data_slider.append(data_each_yr)

C:\Users\Crs\AppData\Roaming\Python\Python39\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [46]:
import plotly.graph_objs as go
layout = dict(geo=dict(scope='europe',
                            projection={'type': 'natural earth'},
                            bgcolor= 'rgba(0,0,0,0)'),
                    margin=dict(l=0,
                                r=0,
                                b=0,
                                t=30,
                                pad=0),
                    paper_bgcolor='rgba(0,0,0,0)',
                    plot_bgcolor='rgba(0,0,0,0)')
 
fig_choropleth = go.Figure(data=data_slider, layout=layout)
fig_choropleth.update_geos(showcoastlines=False, showsubunits=False,showframe=False)

In [32]:
gkk = deaths[deaths['euro']==1].groupby(['euro', 'Entity','drug'])['total'].sum().reset_index()
gkk['perc'] = 100 * gkk['total'] / gkk.groupby(['euro', 'Entity'])['total'].transform('sum')
gkk

,euro,Entity,drug,total,perc
0,1,Austria,Amphetamine,85,1.754024
1,1,Austria,Cocaine,67,1.382584
2,1,Austria,Opioid,4327,89.290136
3,1,Austria,Other,367,7.573256
4,1,Belgium,Amphetamine,238,4.817814
...,...,...,...,...,...
103,1,Spain,Other,4129,17.480948
104,1,Sweden,Amphetamine,607,7.878001
105,1,Sweden,Cocaine,288,3.737833
106,1,Sweden,Opioid,5515,71.576898


In [50]:
country = ['Afghanistan']

In [58]:
fig2 = px.line(deaths[deaths["Entity"].isin(country)], x = "Year", y='total', color='drug')  

In [59]:
fig2 = fig2.update_layout({'margin': dict(t=0, l=0, r=0, b=0),
                               'paper_bgcolor': '#F9F9F8',
                               'font_color': '#363535'})

In [60]:
fig2

In [17]:
x=deaths['Deaths - Opioid use disorders - Sex: Both - Age: All Ages (Number)']
x

0        73
1        80
2        93
3       104
4       112
       ... 
6835     77
6836     81
6837     85
6838     88
6839     92
Name: Deaths - Opioid use disorders - Sex: Both - Age: All Ages (Number), Length: 6840, dtype: int64